In [2]:
# ask_rag.py

from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
import requests
from fastapi.responses import StreamingResponse
import json

# === CONFIGURATION ===
LLM_API_URL = "http://localhost:11434/api/generate" 
COLLECTION_NAME = "rgpd_chunks"
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333

# === INITIALISATION ===
print("🚀 Chargement du modèle d'embedding...")
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

print("🧠 Connexion à Qdrant...")
qdrant = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

def stream_llm(context, question):
    """Envoie un prompt au modèle LLaMA3 via l'API d'Ollama."""
    prompt = f"""
Voici des extraits du RGPD :
{context}

En te basant uniquement sur ces extraits, réponds précisément à la question suivante :
{question}
"""
    def generate():
        with requests.post(
            LLM_API_URL,
            json={
                "model": "llama3:8b-q4_0",
                "prompt": prompt,
                "temperature": 0.2,
                "stream": True
            },
            stream=True
        ) as reponse:
            for line in response.iter_lines():
                if line:
                    try:
                        chunk = json.loads(line.decode("utf-8"))["response"]
                        yield chunk
                    except:
                        pass
    return StreamingResponse(generate(), media_type="text/plain")

        # try:
        #     return response.json()["response"]
        # except Exception as e:
        #     print(response.text)
        #     return f"Erreur de LLM : {e}"


def retrieve_context(query, top_k=3):
    """Effectue une recherche sémantique dans Qdrant et retourne les textes associés."""
    query_vector = embedder.encode(query)
    results = qdrant.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_vector,
        limit=top_k
    )
    return "\n---\n".join([r.payload["text"][:300] for r in results])

# === INTERFACE UTILISATEUR ===
if __name__ == "__main__":
    print("🧑‍💼 Système RAG RGPD prêt. Pose ta question (ou tape 'exit') 👇\n")

    while True:
        question = input("❓> ").strip()
        if question.lower() in ["exit", "quit"]:
            break

        print("🔍 Recherche des documents pertinents...")
        context = retrieve_context(question)

        print("\n🤖 Génération de la réponse...\n")
        response = stream_llm(context, question)
        print("💬 Réponse :\n")
        print(response)
        print("\n" + "=" * 60 + "\n")


🚀 Chargement du modèle d'embedding...
🧠 Connexion à Qdrant...
🧑‍💼 Système RAG RGPD prêt. Pose ta question (ou tape 'exit') 👇

🔍 Recherche des documents pertinents...


/tmp/ipykernel_33634/882255286.py:61: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(



🤖 Génération de la réponse...

💬 Réponse :



🔍 Recherche des documents pertinents...

🤖 Génération de la réponse...

💬 Réponse :



🔍 Recherche des documents pertinents...

🤖 Génération de la réponse...

💬 Réponse :



🔍 Recherche des documents pertinents...

🤖 Génération de la réponse...

💬 Réponse :



🔍 Recherche des documents pertinents...

🤖 Génération de la réponse...

💬 Réponse :



